In [ ]:
from datasets import load_dataset
import pandas as pd
import os
import subprocess

# Get the default gateway IP dynamically
try:
    hostip = subprocess.check_output("ip route | grep default | awk '{print $3}'", shell=True).decode().strip()
    port = 7890
    PROXY_HTTP = f"http://{hostip}:{port}"
    
    # Set proxy environment variables
    os.environ['http_proxy'] = PROXY_HTTP
    os.environ['https_proxy'] = PROXY_HTTP
    print(f"Using proxy: {PROXY_HTTP}")
except Exception as e:
    print(f"Could not set proxy: {e}")

# Load the dataset
dataset = load_dataset("neuralwork/arxiver")

# Convert to pandas DataFrame for easier viewing
df = dataset['train'].to_pandas()

# Show the first 5 papers with title and abstract
# print(df['markdown'].iloc[0])

# df[['markdown']].to_csv('raw_paper_data/markdown_only.csv', index=False)
type(df['markdown'].values)

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
separators = ["\n\n", "\n", " ", "", "."]

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(separators=separators, chunk_size=1000, chunk_overlap=5)

# Split your docs into texts
texts_paper = text_splitter.create_documents(df['markdown'].values)

# Embeddings
hg_embeddings = HuggingFaceEmbeddings()

In [ ]:
from langchain_community.vectorstores import LanceDB
import lancedb

# Create a directory for the database if it doesn't exist
os.makedirs("docs/lancedb_rag", exist_ok=True)

# Connect to a database (creates it if it doesn't exist)
db = lancedb.connect("docs/lancedb_rag")

# Create a LanceDB vector store
langchain_lancedb = LanceDB.from_documents(
    documents=texts_paper,
    embedding=hg_embeddings,
    connection=db,
    table_name="humanizer_data"
)

In [ ]:
from langchain_community.vectorstores import Chroma
persist_directory = 'docs/chroma_rag/'
langchain_chroma = Chroma.from_documents(
    documents=texts_paper,
    collection_name="humanizer_data",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()
persist_directory = 'docs/chroma_rag/'
langchain_chroma = Chroma(
    collection_name="humanizer_data",
    embedding_function=hg_embeddings,
    persist_directory=persist_directory
)

/tmp/ipykernel_24869/1887491695.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  langchain_chroma = Chroma(


In [5]:
langchain_chroma.similarity_search('The authors state "updating the “Break List” (delta) and “Working List” (omega) in Algorithm 1', k = 5)

[Document(metadata={}, page_content='The algorithm performs the following steps:'),
 Document(metadata={}, page_content='**Algorithm 2** Details of the step (a) in Block 2 of the Algorithm 1'),
 Document(metadata={}, page_content='error & refinement) prompts. The details are shown in Algorithm 2.'),
 Document(metadata={}, page_content='The subsequent phases of the process iteratively flip each bit in the partition vector and update \\(MinCost\\) if the cost of the new partition vector is lower. This process is repeated until all bits have been flipped. This marks the completion of one iteration or pass through Algorithm 1. Once a pass is completed, Algorithm 1 prepares the initial partition vector for the next iteration from the partition vector with the lowest cost given by \\(MinCost\\) and the initial partition vector, then applies a crossover operation using Algorithm 4 to generate the new initial partition vector for the next pass.'),
 Document(metadata={}, page_content='Before pr